In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from io import StringIO
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\delete\International Agency for Research on Cancer'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\International Agency for Research on Cancer"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://monographs.iarc.who.int/list-of-classifications")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="table_wrapper"]/div[1]/button[2]/span').click()
time.sleep(10)
driver.close()

In [17]:
path = 'Y:\\delete\\International Agency for Research on Cancer'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    dataset = pd.read_csv(f)
dataset = dataset[['CAS No.', 'Group']]
dataset = dataset[dataset['CAS No.'].notna()]
dataset = dataset.reset_index(drop = True)
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dataset.rename(columns = {'CAS No.':'CAS Number'}, inplace = True)

,CAS Number,Group
0,100-00-5,2B
1,100-17-4,2B
2,100-40-3,2B
3,100-41-4,2B
4,100-42-5,2A
...,...,...
888,99-56-9,3
889,99-57-0,3
890,99-59-2,3
891,99-80-9,3


In [20]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
#CAS['value'] = ''
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

,CAS Number
0,14807-96-6
1,7631-86-91
2,9002-84-0
3,65-85-0
4,100-00-5


In [29]:
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.fillna('4 or not on the list', inplace=True)
#del result['value']

,CAS Number,Group
0,100-00-5,2B
1,100-17-4,2B
2,100-40-3,2B
3,100-41-4,2B
4,100-42-5,2A
...,...,...
890,99-59-2,3
891,99-80-9,3
892,99-97-8,2B
893,7631-86-91,4 or not on the list


In [30]:
result['Jurisdiction'] = 'International'
result['RegulatoryAgency'] = 'International Agency for Research on Cancer (IARC)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Carcinogenicity'

result['Description'] = 'Probably not carcinogenic to humans'
result.loc[result['Group'] == '1', 'Description'] = 'Carcinogenic to humans'
result.loc[result['Group'] == '2A', 'Description'] = 'Probably carcinogenic to humans'
result.loc[result['Group'] == '2B', 'Description'] = 'Possibly carcinogenic to humans'
result.loc[result['Group'] == '3', 'Description'] = 'Unclassifiable as to carcinogenicity to humans'

result['HazardClassification'] = 'Very Low'
result.loc[result['Group'] == '1', 'HazardClassification'] = 'Very High'
result.loc[result['Group'] == '2A', 'HazardClassification'] = 'High'
result.loc[result['Group'] == '2B', 'HazardClassification'] = 'Moderate'
result.loc[result['Group'] == '3', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = result['Group']

result['SourceName'] = 'International Agency for Research on Cancer'
result['SourceURL'] = 'https://monographs.iarc.who.int/list-of-classifications'

result = result.drop(['Group'], axis=1)
result.rename(columns = {'CAS No.':'CAS Number'}, inplace = True)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,100-00-5,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Possibly carcinogenic to humans,Moderate,2B,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
1,100-17-4,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Possibly carcinogenic to humans,Moderate,2B,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
2,100-40-3,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Possibly carcinogenic to humans,Moderate,2B,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
3,100-41-4,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Possibly carcinogenic to humans,Moderate,2B,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
4,100-42-5,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Probably carcinogenic to humans,High,2A,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
...,...,...,...,...,...,...,...,...,...,...
890,99-59-2,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Unclassifiable as to carcinogenicity to humans,Low,3,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
891,99-80-9,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Unclassifiable as to carcinogenicity to humans,Low,3,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
892,99-97-8,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Possibly carcinogenic to humans,Moderate,2B,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
893,7631-86-91,International,International Agency for Research on Cancer (I...,Health,Carcinogenicity,Probably not carcinogenic to humans,Very Low,4 or not on the list,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [31]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:22:43.358263
